In [ ]:
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import emcee
from scipy.stats import beta
import matplotlib.pyplot as plt
import numpy as np

## **Test an estimation on real data**

In [ ]:
# Load the CSV file
file_path = '/Users/echo/Downloads/df_germany.csv'  # Update this path to where your file is located
df = pd.read_csv(file_path)
df = df[df["brand"]=="absolut"]

# Define the dependent variable
Y_t = df['volume_so_off'].values


# df['brand'] = df['brand'].astype('category').cat.codes

# Define the independent variables for X_t and Z_t
# Adjust these columns based on actual needs
X_t = df[['relative_gap_to_90th_price_off_off', 'off_trade_visibility_off', 'digital_off', 'digital_social_media_off',"out_of_home_off",
          'television_off', 'brand_experience_off']].values
Z_t = df[['distribution_off_off', 'discount_price_comp_to_pr_off_off']].values

## **Preliminary experiments**

## **Here we need an initial guess and use a naive estimation of the parameters**

In [ ]:
def estimation_eta_zeta(X_t,Z_t,Y_t):
  XZ_t = np.hstack((X_t, Z_t))
  # Fit the linear regression model
  model_XZ = LinearRegression().fit(XZ_t, Y_t_normalized)
  eta_zeta = model_XZ.coef_
  # Separate eta and zeta
  eta = eta_zeta[:X_t.shape[1]]
  zeta = eta_zeta[X_t.shape[1]:]
  return([eta,zeta])

Second experiments

In [ ]:

# # Initial parameter values
# initial_params = [0.5] + [0.0] * (X_t.shape[1] + Z_t.shape[1])
# Original parameters
original_G = 0.7
original_eta = eta
original_zeta = zeta

# Add Gaussian noise to the initial parameters
initial_G = original_G + np.random.normal(0, 0.01)
initial_eta = original_eta + np.random.normal(0, 0.01, size=original_eta.shape)
initial_zeta = original_zeta + np.random.normal(0, 0.01, size=original_zeta.shape)

# Combine the initial parameters
initial_params = [initial_G] + list(initial_eta) + list(initial_zeta)

# Perform gradient descent
optimized_params = gradient_descent(Y_t, X_t, Z_t, initial_params)

# Extract optimized parameters
G, *coeffs = optimized_params

eta = np.array(coeffs[:X_t.shape[1]])
zeta = np.array(coeffs[X_t.shape[1]:])

print("new G", G)
print("new eta", eta)
print("new zeta", zeta)

# Define the DLM function according to the formula
def dlm_model_optimized(X_t, Z_t, eta, zeta, G, T):
    theta_t = np.zeros(T)
    predicted_Y = np.zeros(T)
    for t in range(T):
        if t > 0:
            theta_t[t] = G * theta_t[t-1] + np.dot(Z_t[t-1], zeta / 2)
        predicted_Y[t] = theta_t[t] + np.dot(X_t[t], eta) + np.dot(Z_t[t], zeta / 2)

    return predicted_Y

# Define the number of time steps
T = len(Y_t)

# Predict Y_t using the optimized DLM model
optimized_predicted_Y = dlm_model_optimized(X_t, Z_t, eta, zeta, G, T)

# Convert the results to a DataFrame for visualization
results_df = pd.DataFrame({
    'Actual_Y': scaler_Y.inverse_transform(Y_t.reshape(-1, 1)).flatten(),
    'Optimized_Predicted_Y': scaler_Y.inverse_transform(optimized_predicted_Y.reshape(-1, 1)).flatten()
})

print(results_df)

# Evaluate the model performance
mse = mean_squared_error(results_df['Actual_Y'], results_df['Optimized_Predicted_Y'])
print(f'Mean Squared Error: {mse}')

# Plot actual vs predicted
plt.figure(figsize=(10, 5))
plt.plot(results_df['Actual_Y'], label='Actual Y_t')
plt.plot(results_df['Optimized_Predicted_Y'], label='Optimized Predicted Y_t', linestyle='--')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Volume Sold')
plt.title('Actual vs Optimized Predicted Y_t')
plt.show()

Third experiments

In [ ]:
#for i...:
# y_t[i]=dlm_model(X_t, Z_t, eta, zeta, G, T)
#param[i]=estimation_gradient_descent(Y_t, X_t, Z_t, initial_params, learning_rate=0.001, n_iterations=500)